# Extraction referentiel France Travail

Ce script a pour obhectif d'extraire le référentiel associées aux offres d'emploi proposées par France Travail et de le stocker en local dans un fichier Excel.

Comment ?
1. Récupération du token permettant d'utiliser l'API de France Travail.
2. Sur la base de critères spécifiques (i.e l'intégralité des référentiels existants), lancement d'une requête pour obtenir les données correspondantes.
3. Une fois les données trouvées, une sauvegarde en local est exécutée et les offres sont stockées dans un fichier (Excel).

Les URL utiles sont :
- https://francetravail.io/data/api/offres-emploi
- https://francetravail.io/data/api/offres-emploi/documentation#/

## Imports

In [16]:
import http.client
import requests
import json
import pandas as pd
import os

# Procédure

## Configuration

In [17]:
# ---------------------------
# CONFIGURATION
# ---------------------------
# France Travail (ex-Pôle emploi)
# FT_CLIENT_ID = Defined in .env
# FT_CLIENT_SECRET = Defined in .env
# FT_SCOPE = Defined in .env

##################  VARIABLES  ##################
FT_CLIENT_ID = os.environ.get("FT_CLIENT_ID")
FT_CLIENT_SECRET = os.environ.get("FT_CLIENT_SECRET")
FT_SCOPE = os.environ.get("FT_SCOPE")

## Paramètres

In [18]:
# ---------------------------
# PARAMETRES
# ---------------------------

# Non applicable ici.

## Authentification France Travail

In [19]:
# ---------------------------
# AUTH FRANCE TRAVAIL
# ---------------------------
def get_ft_token():
    url = "https://entreprise.pole-emploi.fr/connexion/oauth2/access_token?realm=/partenaire"
    data = {
        "grant_type": "client_credentials",
        "client_id": FT_CLIENT_ID,
        "client_secret": FT_CLIENT_SECRET,
        "scope": FT_SCOPE,
    }
    r = requests.post(url, data=data)
    r.raise_for_status()
    return r.json()["access_token"]

### Lancement requête API France Travail

In [20]:
# API CALL FRANCE TRAVAIL
# ---------------------------
def fetch_france_travail_referentiel(token):
    referential = ["appellations",
        "nafs",
        "communes",
        "continents",
        "departements",
        "domaines",
        "langues",
        "metiers",
        "naturesContrats",
        "niveauxFormations",
        "pays",
        "permis",
        "regions",
        "secteursActivites",
        "themes",
        "typesContrats"]
    headers = {"Authorization": f"Bearer {token}"}


    # Export vers CSV
    path = "../data/raw_data/"
    file_name = "referentiel_France_Travail.xlsx"
 
    # Save into different sheets of the same Excel file
    with pd.ExcelWriter(path + file_name) as writer:
        for ref in referential:
            url = f"https://api.francetravail.io/partenaire/offresdemploi/v2/referentiel/{ref}"
            r = requests.get(url, headers=headers)
            r.raise_for_status()
            data = r.json()
            df = pd.DataFrame(data)
            df.to_excel(writer, sheet_name=f"{ref}", index=False)   
    print(f"Référentiel exporté dans {file_name} ✅")

### Procédure principale

In [21]:
# ---------------------------
# MAIN
# ---------------------------
if __name__ == "__main__":
    print("Authentification France Travail...")
    token = get_ft_token()

    print("Récupération du référentiel France Travail...")
    fetch_france_travail_referentiel(token)

Authentification France Travail...
Récupération du référentiel France Travail...
Référentiel exporté dans referentiel_France_Travail.xlsx ✅
